# Run Gold Layer Processing Only

This notebook creates fact and dimension tables from existing silver tables.

## 1. Install the Wheel Files (if not already installed)

In [1]:
# Install the wheel files (skip if already installed)
%pip install /lakehouse/default/Files/unified_etl_core-1.0.0-py3-none-any.whl
%pip install /lakehouse/default/Files/unified_etl_connectwise-1.0.0-py3-none-any.whl

StatementMeta(, 101a9a68-9f17-4a04-9c35-5f19f96d4c60, 9, Finished, Available, Finished)

Processing /lakehouse/default/Files/unified_etl_core-1.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 8.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-34cb494c-f51e-4915-8a4b-eafd69376a33
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: pyparsing
    Found existing installati

## 2. Check Existing Silver Tables

Verify we have silver tables to process:

# List silver tables in silver schema
silver_tables = spark.sql("SHOW TABLES IN silver").collect()
print(f"Found {len(silver_tables)} tables in silver schema:")

# Build silver table mapping and show counts
silver_table_mapping = {}
for row in silver_tables:
    table_name = row.tableName
    full_table_name = f"silver.{table_name}"
    count = spark.sql(f"SELECT COUNT(*) FROM {full_table_name}").collect()[0][0]
    print(f"  - {full_table_name}: {count:,} rows")
    
    # Extract entity name (e.g., silver_cw_agreement -> agreement)
    entity_name = table_name.replace("silver_cw_", "")
    silver_table_mapping[entity_name] = full_table_name

print(f"\nEntities to process: {list(silver_table_mapping.keys())}")

## 3. Run Gold Layer Processing

Create fact tables with business logic:

In [3]:
import logging

from unified_etl_core.main import run_etl_pipeline

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s -%(levelname)s - %(message)s"
)

print("🥇 Starting Gold Layer Processing...")
print("This will create fact tables with business logic")

# Build table mappings for schema-enabled lakehouse
table_mappings = {
    "silver": silver_table_mapping,  # From previous cell
    "gold": {},
}

# Add gold table mappings - these will be used for the fact tables created
for entity_name in silver_table_mapping.keys():
    table_mappings["gold"][f"fact_{entity_name}"] = f"gold.gold_cw_fact_{entity_name}"

# Also add mappings for specialized fact tables
table_mappings["gold"]["fact_agreement_period"] = "gold.gold_cw_fact_agreement_period"
table_mappings["gold"]["fact_agreement_summary"] = "gold.gold_cw_fact_agreement_summary"
table_mappings["gold"]["fact_invoice_line"] = "gold.gold_cw_fact_invoice_line"
table_mappings["gold"]["fact_invoice_header"] = "gold.gold_cw_fact_invoice_header"
table_mappings["gold"]["fact_invoice_period"] = "gold.gold_cw_fact_invoice_period"

# Create entity configurations for gold processing with ConnectWise-specific settings
entity_configs = {
    "agreement": {
        "source": "connectwise",
        "surrogate_keys": [{"name": "AgreementSK", "business_keys": ["id"]}],
        "business_keys": [{"name": "AgreementBusinessKey", "source_columns": ["id"]}],
        "calculated_columns": {
            "estimated_monthly_revenue": "CASE WHEN applicationUnits = 'Amount' THEN COALESCE(applicationLimit, 0) ELSE 0 END"
        },
        # ConnectWise-specific transform settings
        "gold_transforms": {
            "fact_agreement_period": {
                "date_spine": {"start": "2020-01-01", "frequency": "month"},
                "metrics": [
                    "is_active_period",
                    "is_new_agreement",
                    "is_churned_agreement",
                    "monthly_revenue",
                    "prorated_revenue",
                    "months_since_start",
                    "revenue_change",
                    "cumulative_revenue",
                ],
                "keys": [
                    {
                        "name": "AgreementPeriodSK",
                        "type": "hash",
                        "source_columns": ["id", "period_start"],
                    }
                ],
            },
            "fact_agreement_summary": {
                "metrics": [
                    "lifetime_days",
                    "lifetime_months",
                    "estimated_lifetime_value",
                    "actual_total_revenue",
                    "actual_avg_monthly_revenue",
                    "active_periods",
                ],
                "keys": [{"name": "AgreementSummarySK", "type": "hash", "source_columns": ["id"]}],
            },
        },
    },
    "invoice": {
        "source": "connectwise",
        "surrogate_keys": [{"name": "InvoiceSK", "business_keys": ["id"]}],
        "business_keys": [{"name": "InvoiceBusinessKey", "source_columns": ["id"]}],
        "calculated_columns": {},
        # ConnectWise-specific invoice transform settings
        "enable_period_facts": True,
        "period_type": "month",
    },
}

# Add basic configs for other entities without specialized transforms
for entity in silver_table_mapping.keys():
    if entity not in entity_configs:
        entity_configs[entity] = {
            "source": "connectwise",
            "surrogate_keys": [{"name": f"{entity.title()}SK", "business_keys": ["id"]}],
            "business_keys": [{"name": f"{entity.title()}BusinessKey", "source_columns": ["id"]}],
            "calculated_columns": {},
        }

config = {"entities": entity_configs}

print(f"Processing entities: {list(entity_configs.keys())}")
print("Will use ConnectWise-specific transforms for agreement and invoice entities")

# Run Gold layer with proper table mappings
run_etl_pipeline(
    integrations=["connectwise"], layers=["gold"], config=config, table_mappings=table_mappings
)

StatementMeta(, 101a9a68-9f17-4a04-9c35-5f19f96d4c60, 12, Finished, Available, Finished)

🥇 Starting Gold Layer Processing...
This will create fact tables with business logic
Processing entities: ['agreement', 'invoice', 'expenseentry', 'productitem', 'timeentry']
Will use ConnectWise-specific transforms for agreement and invoice entities


2025-05-28 10:47:25,002 - root -INFO - ✅ Integration 'connectwise' detected and loaded
2025-05-28 10:47:25,003 - root -INFO - ⚠️ Integration 'businesscentral' not available (package not installed)
2025-05-28 10:47:25,004 - root -INFO - ⚠️ Integration 'jira' not available (package not installed)
2025-05-28 10:47:25,004 - root -INFO - Running ETL pipeline for integrations: ['connectwise']
2025-05-28 10:47:25,005 - root -INFO - Processing layers: ['gold']
2025-05-28 10:47:25,005 - root -INFO - Processing integration: connectwise
2025-05-28 10:47:25,006 - root -INFO - Running gold layer for connectwise
2025-05-28 10:47:25,011 - root -INFO - Using ConnectWise-specific transforms
2025-05-28 10:47:25,152 - root -INFO - Using ConnectWise agreement-specific transforms
2025-05-28 10:47:34,239 - root -INFO - Created fact table gold.gold_cw_fact_agreement_period
2025-05-28 10:47:39,063 - root -INFO - Created fact table gold.gold_cw_fact_agreement_summary
2025-05-28 10:47:39,531 - root -INFO - Usin

## 4. Verify Gold Tables

In [4]:
# Check gold tables
gold_tables = spark.sql("SHOW TABLES IN gold").collect()
print(f"Found {len(gold_tables)} tables in gold schema:")

for row in gold_tables:
    table_name = row.tableName
    full_table_name = f"gold.{table_name}"
    df = spark.sql(f"SELECT * FROM {full_table_name}")
    count = df.count()
    print(f"\n{full_table_name}:")
    print(f"  Rows: {count:,}")
    print(f"  Columns: {len(df.columns)}")

    # Show a few sample columns with data
    sample_cols = df.columns[:5]
    if "id" in df.columns and "id" not in sample_cols:
        sample_cols.append("id")
    if any("SK" in col for col in df.columns):
        sk_col = next(col for col in df.columns if "SK" in col)
        if sk_col not in sample_cols:
            sample_cols.append(sk_col)

    print(f"  Sample data ({', '.join(sample_cols)}):")
    df.select(*sample_cols).show(5, truncate=False)

StatementMeta(, 101a9a68-9f17-4a04-9c35-5f19f96d4c60, 13, Finished, Available, Finished)

Found 9 tables in gold schema:

gold.gold_cw_fact_agreement_period:
  Rows: 72,925
  Columns: 29
  Sample data (period_start, period_end, year, month, quarter, id, AgreementPeriodSK):
+------------+----------+----+-----+-------+---+----------------------------------------------------------------+
|period_start|period_end|year|month|quarter|id |AgreementPeriodSK                                               |
+------------+----------+----+-----+-------+---+----------------------------------------------------------------+
|2020-01-01  |2020-01-31|2020|1    |1      |920|9a58b7dd2318fcfaf7dfc8ee43e6f281e2272f5af9f632523292a64e17d4ff8f|
|2020-03-01  |2020-03-31|2020|3    |1      |920|81d45f836f21d27e4ea584e97643a586f45b7a014b4112f79e0ffe28daeac1eb|
|2020-05-01  |2020-05-31|2020|5    |2      |920|90ec29fc07a1f87e075bf2b3c0eacd2e933ae3032246566e6b401790e0086b75|
|2020-07-01  |2020-07-31|2020|7    |3      |920|993067ccadbe2231ceb66c844e6c851b011ce9b1c60d22e268d4272de2e992b3|
|2020-08-01  |2020

## 5. Performance Summary

In [5]:
print("\n📊 Gold Layer Processing Summary")
print("=" * 50)

# Get row counts for each schema
for schema in ["bronze", "silver", "gold"]:
    tables = spark.sql(f"SHOW TABLES IN {schema}").collect()
    total_rows = 0
    table_details = []

    for row in tables:
        table_name = row.tableName
        full_name = f"{schema}.{table_name}"
        count = spark.sql(f"SELECT COUNT(*) FROM {full_name}").collect()[0][0]
        total_rows += count
        table_details.append((table_name, count))

    print(f"\n{schema.upper()} Schema:")
    print(f"  Tables: {len(tables)}")
    print(f"  Total rows: {total_rows:,}")
    for table, count in sorted(table_details):
        print(f"    - {table}: {count:,} rows")

print("\n✅ Gold layer complete - ready for reporting and analytics!")

StatementMeta(, 101a9a68-9f17-4a04-9c35-5f19f96d4c60, 14, Finished, Available, Finished)


📊 Gold Layer Processing Summary

BRONZE Schema:
  Tables: 5
  Total rows: 969,126
    - bronze_cw_agreement: 1,933 rows
    - bronze_cw_expenseentry: 14,188 rows
    - bronze_cw_invoice: 37,294 rows
    - bronze_cw_productitem: 379,748 rows
    - bronze_cw_timeentry: 535,963 rows

SILVER Schema:
  Tables: 5
  Total rows: 969,126
    - silver_cw_agreement: 1,933 rows
    - silver_cw_expenseentry: 14,188 rows
    - silver_cw_invoice: 37,294 rows
    - silver_cw_productitem: 379,748 rows
    - silver_cw_timeentry: 535,963 rows

GOLD Schema:
  Tables: 9
  Total rows: 1,580,018
    - gold_cw_fact_agreement_period: 72,925 rows
    - gold_cw_fact_agreement_summary: 1,933 rows
    - gold_cw_fact_expenseentry: 14,188 rows
    - gold_cw_fact_invoice: 37,294 rows
    - gold_cw_fact_invoice_header: 37,294 rows
    - gold_cw_fact_invoice_line: 463,379 rows
    - gold_cw_fact_invoice_period: 37,294 rows
    - gold_cw_fact_productitem: 379,748 rows
    - gold_cw_fact_timeentry: 535,963 rows

✅ Gold 

In [7]:
# Print schemas for all the gold fact tables
print("🥇 GOLD LAYER SCHEMAS")
print("=" * 50)

print("\n📊 AGREEMENT FACTS")
print("-" * 30)
print("Agreement Period Facts (Monthly grain):")
spark.table("gold.gold_cw_fact_agreement_period").printSchema()

print("\nAgreement Summary Facts (Lifetime grain):")
spark.table("gold.gold_cw_fact_agreement_summary").printSchema()

print("\n💰 INVOICE FACTS")
print("-" * 30)
print("Invoice Line Facts (Most detailed grain - from time entries + products):")
spark.table("gold.gold_cw_fact_invoice_line").printSchema()

print("\nInvoice Header Facts (Document grain):")
spark.table("gold.gold_cw_fact_invoice_header").printSchema()

print("\nInvoice Period Facts (Revenue recognition grain):")
spark.table("gold.gold_cw_fact_invoice_period").printSchema()

print("\n📋 OTHER ENTITY FACTS")
print("-" * 30)
print("Expense Entry Facts:")
spark.table("gold.gold_cw_fact_expenseentry").printSchema()

print("\nProduct Item Facts:")
spark.table("gold.gold_cw_fact_productitem").printSchema()

print("\nTime Entry Facts:")
spark.table("gold.gold_cw_fact_timeentry").printSchema()

print("\nGeneric Invoice Facts (fallback):")
spark.table("gold.gold_cw_fact_invoice").printSchema()

print("\n🎯 GOLD LAYER SUMMARY")
print("=" * 50)
print("Total Gold Tables: 9")
print("Total Rows: 1,580,018")
print("\nSpecialized ConnectWise Transforms:")
print("✅ Agreement period facts (72,925 rows) - Monthly MRR tracking")
print("✅ Agreement summary facts (1,933 rows) - Lifetime metrics")
print("✅ Invoice line facts (463,379 rows) - Created from time entries + products")
print("✅ Invoice header facts (37,294 rows) - Document-level aggregations")
print("✅ Invoice period facts (37,294 rows) - Revenue recognition")
print("\nGeneric Transforms:")
print("📝 Expense entry facts (14,188 rows)")
print("📦 Product item facts (379,748 rows)")
print("⏰ Time entry facts (535,963 rows)")
print("📄 Invoice facts (37,294 rows) - Generic fallback")

StatementMeta(, 101a9a68-9f17-4a04-9c35-5f19f96d4c60, 16, Finished, Available, Finished)

🥇 GOLD LAYER SCHEMAS

📊 AGREEMENT FACTS
------------------------------
Agreement Period Facts (Monthly grain):
root
 |-- period_start: date (nullable = true)
 |-- period_end: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- agreementStatus: string (nullable = true)
 |-- typeId: integer (nullable = true)
 |-- typeName: string (nullable = true)
 |-- companyId: integer (nullable = true)
 |-- contactId: integer (nullable = true)
 |-- billingCycleId: integer (nullable = true)
 |-- billAmount: double (nullable = true)
 |-- applicationUnits: string (nullable = true)
 |-- effective_start: date (nullable = true)
 |-- effective_end: timestamp (nullable = true)
 |-- cancelledFlag: boolean (nullable = true)
 |-- customFields: string (nullable = true)
 |-- is_active_period: boolean (nullable = true)
 |-- is_new_agreement: boolean (nulla

In [10]:
print("🎯 Creating BI Materialized Views from Gold Facts")
print("=" * 50)

# 1. Monthly Revenue Dashboard - Materialized View
spark.sql("""
CREATE OR REPLACE MATERIALIZED VIEW gold.mv_monthly_revenue_dashboard AS
SELECT
    DATE_FORMAT(period_start, 'yyyy-MM') as revenue_month,
    year,
    month,
    quarter,
    COUNT(DISTINCT id) as active_agreements,
    SUM(monthly_revenue) as total_mrr,
    SUM(CASE WHEN is_new_agreement THEN monthly_revenue ELSE 0 END) as new_mrr,
    SUM(CASE WHEN is_churned_agreement THEN monthly_revenue ELSE 0 END) as churned_mrr,
    COUNT(CASE WHEN is_new_agreement THEN 1 END) as new_agreement_count,
    COUNT(CASE WHEN is_churned_agreement THEN 1 END) as churned_agreement_count,
    AVG(monthly_revenue) as avg_agreement_value
FROM gold.gold_cw_fact_agreement_period
WHERE is_active_period = true
GROUP BY DATE_FORMAT(period_start, 'yyyy-MM'), year, month, quarter
""")

# 2. Customer Revenue Analysis - Materialized View
spark.sql("""
CREATE OR REPLACE MATERIALIZED VIEW gold.mv_customer_revenue_analysis AS
SELECT
    h.companyId,
    h.companyName,
    h.billToCompanyName,
    COUNT(DISTINCT h.id) as invoice_count,
    SUM(h.total) as total_invoiced,
    SUM(h.totalMargin) as total_margin,
    AVG(h.avgMarginPercentage) as avg_margin_pct,
    SUM(h.lineItemCount) as total_line_items,
    COUNT(DISTINCT l.agreementId) as agreement_count,
    -- Revenue by type
    SUM(CASE WHEN l.isService THEN l.lineAmount ELSE 0 END) as service_revenue,
    SUM(CASE WHEN l.isProduct THEN l.lineAmount ELSE 0 END) as product_revenue,
    SUM(CASE WHEN l.isAgreement THEN l.lineAmount ELSE 0 END) as agreement_revenue,
    -- Time-based metrics
    MIN(h.date) as first_invoice_date,
    MAX(h.date) as last_invoice_date,
    DATEDIFF(MAX(h.date), MIN(h.date)) as customer_lifetime_days
FROM gold.gold_cw_fact_invoice_header h
LEFT JOIN gold.gold_cw_fact_invoice_line l ON h.InvoiceSK = l.InvoiceSK
GROUP BY h.companyId, h.companyName, h.billToCompanyName
HAVING total_invoiced > 0
""")

# 3. Resource Utilization - Materialized View
spark.sql("""
CREATE OR REPLACE MATERIALIZED VIEW gold.mv_resource_utilization AS
SELECT
    t.memberId,
    t.memberName,
    t.workTypeName,
    t.workRoleName,
    t.companyName,
    t.agreementName,
    t.agreementType,
    DATE_FORMAT(t.timeStart, 'yyyy-MM') as work_month,
    -- Time metrics
    SUM(t.actualHours) as total_hours,
    SUM(t.hoursBilled) as billable_hours,
    SUM(t.actualHours * t.hourlyRate) as total_value,
    AVG(t.hourlyRate) as avg_hourly_rate,
    -- Efficiency metrics
    ROUND(SUM(t.hoursBilled) / SUM(t.actualHours) * 100, 2) as utilization_rate,
    COUNT(DISTINCT t.id) as time_entry_count,
    COUNT(DISTINCT t.companyId) as client_count,
    COUNT(DISTINCT DATE(t.timeStart)) as work_days
FROM gold.gold_cw_fact_timeentry t
WHERE t.actualHours > 0
GROUP BY t.memberId, t.memberName, t.workTypeName, t.workRoleName,
        t.companyName, t.agreementName, t.agreementType,
        DATE_FORMAT(t.timeStart, 'yyyy-MM')
""")

# 4. Agreement Performance - Materialized View
spark.sql("""
CREATE OR REPLACE MATERIALIZED VIEW gold.mv_agreement_performance AS
SELECT
    s.id as agreement_id,
    s.name as agreement_name,
    s.typeName as agreement_type,
    s.companyName,
    s.agreementStatus,
    s.billAmount as monthly_bill_amount,
    s.lifetime_months,
    s.estimated_lifetime_value,
    s.actual_total_revenue,
    s.actual_avg_monthly_revenue,
    s.cumulative_lifetime_revenue,
    -- Performance metrics
    CASE
        WHEN s.estimated_lifetime_value > 0
        THEN ROUND(s.actual_total_revenue / s.estimated_lifetime_value * 100, 2)
        ELSE 0
    END as revenue_realization_pct,
    ROUND(s.actual_avg_monthly_revenue / s.billAmount * 100, 2) as billing_efficiency_pct,
    s.active_periods,
    s.start_date,
    s.endDate as end_date,
    s.cancelledFlag
FROM gold.gold_cw_fact_agreement_summary s
""")

# 5. Revenue Mix Analysis - Materialized View
spark.sql("""
CREATE OR REPLACE MATERIALIZED VIEW gold.mv_revenue_mix_analysis AS
SELECT DATE_FORMAT(l.date, 'yyyy-MM') as revenue_month,l.companyId,
    -- Revenue breakdown
    SUM(CASE WHEN l.isService THEN l.lineAmount ELSE 0 END) as service_revenue,
    SUM(CASE WHEN l.isProduct THEN l.lineAmount ELSE 0 END) as product_revenue,
    SUM(CASE WHEN l.isAgreement THEN l.lineAmount ELSE 0 END) as agreement_revenue,
    SUM(l.lineAmount) as total_revenue,
    -- Mix percentages
    ROUND(SUM(CASE WHEN l.isService THEN l.lineAmount ELSE 0 END) / SUM(l.lineAmount) *
100, 2) as service_pct,
    ROUND(SUM(CASE WHEN l.isProduct THEN l.lineAmount ELSE 0 END) / SUM(l.lineAmount) *
100, 2) as product_pct,
    ROUND(SUM(CASE WHEN l.isAgreement THEN l.lineAmount ELSE 0 END) / SUM(l.lineAmount) *
100, 2) as agreement_pct,
    -- Volume metrics
    COUNT(CASE WHEN l.isService THEN 1 END) as service_line_count,
    COUNT(CASE WHEN l.isProduct THEN 1 END) as product_line_count,
    COUNT(CASE WHEN l.isAgreement THEN 1 END) as agreement_line_count,
    -- Margin analysis
    AVG(CASE WHEN l.isService THEN l.marginPercentage END) as avg_service_margin_pct,
    AVG(CASE WHEN l.isProduct THEN l.marginPercentage END) as avg_product_margin_pct
FROM gold.gold_cw_fact_invoice_line l
GROUP BY DATE_FORMAT(l.date, 'yyyy-MM'), l.companyId
HAVING total_revenue > 0
""")

print("✅ Created 5 BI Materialized Views:")
print("📊 mv_monthly_revenue_dashboard - MRR tracking and growth metrics")
print("👥 mv_customer_revenue_analysis - Customer lifetime value and segmentation")
print("⏰ mv_resource_utilization - Employee productivity and billing efficiency")
print("📋 mv_agreement_performance - Agreement ROI and realization")
print("🎯 mv_revenue_mix_analysis - Service vs Product revenue trends")

print("\n🔍 Quick data quality check:")
print("Let's spot check the monthly revenue dashboard...")

# Quick data validation
display(
    spark.sql("""
    SELECT revenue_month, active_agreements, total_mrr, new_mrr, churned_mrr
    FROM gold.mv_monthly_revenue_dashboard
    ORDER BY revenue_month DESC
    LIMIT 10
""")
)

StatementMeta(, 101a9a68-9f17-4a04-9c35-5f19f96d4c60, 19, Finished, Available, Finished)

🎯 Creating BI Materialized Views from Gold Facts


ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'MATERIALIZED'.(line 2, pos 18)

== SQL ==

CREATE OR REPLACE MATERIALIZED VIEW gold.mv_monthly_revenue_dashboard AS
------------------^^^
SELECT
    DATE_FORMAT(period_start, 'yyyy-MM') as revenue_month,
    year,
    month,
    quarter,
    COUNT(DISTINCT id) as active_agreements,
    SUM(monthly_revenue) as total_mrr,
    SUM(CASE WHEN is_new_agreement THEN monthly_revenue ELSE 0 END) as new_mrr,
    SUM(CASE WHEN is_churned_agreement THEN monthly_revenue ELSE 0 END) as churned_mrr,
    COUNT(CASE WHEN is_new_agreement THEN 1 END) as new_agreement_count,
    COUNT(CASE WHEN is_churned_agreement THEN 1 END) as churned_agreement_count,
    AVG(monthly_revenue) as avg_agreement_value
FROM gold.gold_cw_fact_agreement_period
WHERE is_active_period = true
GROUP BY DATE_FORMAT(period_start, 'yyyy-MM'), year, month, quarter
